In [1]:
import LeakDetector
import pandas as pd
from glob import glob

In [2]:
from device_ids import TV_ID_MAP
from log_analysis import get_crawl_parameter
from crawl_ids import CrawlRokuTop1KMITM
from df_utils import load_df

## Load leaks
- Run the following to detect and pickle leaks
  - `python2 detect_title_leaks.py` 

In [3]:
leaks = load_df(CrawlRokuTop1KMITM, "title_leak")

In [10]:
from log_analysis import add_domain_column
add_domain_column(leaks)

title_leaks = leaks[(leaks.id_type !="Channel name")]
print(title_leaks.channel_id.nunique(), "channels")
title_leaks[['channel_id', 'channel_name', 'id_type', 'encoding', 'search', 'leak_type', 'req_domain']].drop_duplicates()

(9, 'channels')


,channel_id,channel_name,id_type,encoding,search,leak_type,req_domain
0,11055,Newsy,imdb_title_Newsy's Latest Headlines,unencoded,Newsy's Latest Headlines,post_leaks,google-analytics.com
0,163348,WCJB TV-20 News,imdb_title_Lets Go with Livestream,urlencode,Lets Go with Livestream,url_leaks,scorecardresearch.com
0,27536,CBS News,imdb_title_CBSN Live Video,urlencode,CBSN Live Video,url_leaks,scorecardresearch.com
0,47416,1011 News,imdb_title_Live Newscasts,urlencode,Live Newscasts,url_leaks,scorecardresearch.com
0,47425,WEAU News,imdb_title_Live Newscasts,urlencode,Live Newscasts,url_leaks,scorecardresearch.com
0,64346,FilmRise Kids,imdb_title_Barnum,unencoded,Barnum,url_leaks,spotxchange.com
0,90568,KJRH 2 Works for You Tulsa,imdb_title_Sunday Night Forecast,unencoded,Sunday Night Forecast,post_leaks,google-analytics.com
0,90587,News 5 Cleveland WEWS,imdb_title_Freddie Kitchens makes surprise app...,unencoded,Freddie Kitchens makes surprise appearance,post_leaks,google-analytics.com
0,90630,NewsChannel 5 Nashville WTVF,imdb_title_Live: NewsChannel 5 This Morning at 4,unencoded,Live: NewsChannel 5 This Morning at 4,post_leaks,google-analytics.com


In [7]:
title_leaks[title_leaks.search.isin(["Barnum"])].url.iloc[0]

u'http://search.spotxchange.com/vast/2.00/222236?VPI[]=ROKU&VPI[]=MP4&app[bundle]=roku.happykids&app[name]=happykids&app[cat]=IAB6&app[privacypolicy]=1&app[storeurl]=https%3A%2F%2Fchannelstore.roku.com%2Fdetails%2F32614%2Fhappykids&device[ua]=Roku%2FDVP-9.0%20%28519.00E04142A%29&device[devicetype]=7&device[make]=Roku&device[model]=3900X&player_height=1080&player_width=1920&device[ifa]=ded0f0e3-b3aa-59a2-a143-f6c1157a7ae8&ip_addr=128.112.139.195&regs[coppa]=1&device[lmt]=0&token[content_id]=Barnum&token[content_length]=6612&app[id]=64346&token[genre]=64346&custom[app_id]=64346'

In [11]:
from nb_utils import make_latex_table
print(make_latex_table(title_leaks[['channel_name', 'search', 'req_domain']].drop_duplicates(),
    caption="Title leaks in 100 random Roku channels",
    label="roku_title_leaks"))




\begin{table}[H]
%\centering
%\resizebox{\columnwidth}{!}{%
\begin{tabular}{lll}
\toprule
 Channel name                 & Search                                     & Req domain            \\
\midrule
 Newsy                        & Newsy's Latest Headlines                   & google-analytics.com  \\
 WCJB TV-20 News              & Lets Go with Livestream                    & scorecardresearch.com \\
 CBS News                     & CBSN Live Video                            & scorecardresearch.com \\
 1011 News                    & Live Newscasts                             & scorecardresearch.com \\
 WEAU News                    & Live Newscasts                             & scorecardresearch.com \\
 FilmRise Kids                & Barnum                                     & spotxchange.com       \\
 KJRH 2 Works for You Tulsa   & Sunday Night Forecast                      & google-analytics.com  \\
 News 5 Cleveland WEWS        & Freddie Kitchens makes surprise appearance & google-

In [13]:
title_leaks[~title_leaks.adblocked]

,adblocked,adblocked_by_url,category,channel_id,channel_name,cookie,decoded_data,disconnect_blocked,disconnect_blocked_by_url,domain_by_dns,...,rank,referer,req_domain,search,status,tcp_dstport,tcp_stream,time,url,user_agent


In [17]:
title_leaks[['channel_id', 'category']].drop_duplicates()['category'].value_counts()

News & Weather    8
Kids & Family     1
Name: category, dtype: int64

In [23]:
title_leaks[title_leaks.url.str.startswith('https://')]

,adblocked,adblocked_by_url,category,channel_id,channel_name,cookie,decoded_data,disconnect_blocked,disconnect_blocked_by_url,domain_by_dns,...,rank,referer,req_domain,search,status,tcp_dstport,tcp_stream,time,url,user_agent


In [25]:
title_leaks[title_leaks.url.str.startswith('https://')]

,adblocked,adblocked_by_url,category,channel_id,channel_name,cookie,decoded_data,disconnect_blocked,disconnect_blocked_by_url,domain_by_dns,...,rank,referer,req_domain,search,status,tcp_dstport,tcp_stream,time,url,user_agent
